In [22]:
import geojson
from urllib.request import urlopen
from shapely import geometry, wkt
import pathlib
import pandas
import os
import rasterio
from rasterio.merge import merge 
import rasterio.features
import numpy as np
import glob
import matplotlib.pyplot as plt
import geopandas as gpd
from shapely.geometry import Polygon, box
from creodias_finder import query, download
from rasterio.plot import show

shp_fn = '/ncr2420/ANSU/6_Tasks/2110_ET/test/belgium_west_polygon.shp' 
rst_fn = '/ncr2420/ANSU/6_Tasks/2110_ET/test/S2GLC_belgium_west_2017_old.tif' 
out_dir = '/ncr2420/ANSU/6_Tasks/2110_ET/test/'


value_class = 73

In [23]:
def rasterize_vector(gdf, profile, **kwargs):
    gdf = gdf.to_crs(profile['crs'])
    geoms = [g.__geo_interface__ for g in gdf.geometry.values]
    mask = rasterio.features.rasterize(
            geoms,
            out_shape=(profile['width'],profile['height']),
            transform=profile['transform'], fill= 0,
            **kwargs)
    return mask

def mask_raster_with_vector(rasterfile, vectorfile, path):
    with rasterio.open(rasterfile) as src:
        profile = src.profile.copy()
            
    gdf = gpd.read_file(vectorfile)
   
    mask = rasterize_vector(gdf, profile)
    out_file = str(path + "rasterize.tif")
    with rasterio.open(out_file, "w", **profile) as dst:
        dst.write(mask.astype(rasterio.uint8), 1)
    
    return out_file


In [24]:
rasterized_vector = mask_raster_with_vector(rst_fn,shp_fn, out_dir)

In [25]:

def reclassified(raster_top, raster_down, out_dir=None):
    print(raster_top)
    with rasterio.open(raster_down) as src_down:
        data_down = src_down.read(1)
            
    # Create classification masks
    with rasterio.open(raster_top) as src_top:
        data_top = src_top.read(1)
        height, width = data_top.shape  
        profile = src_top.profile
        data = np.where(data_top==1, value_class, data_down)

    if out_dir != None:
        out_name = os.path.join(out_dir, f'reclassfied_LC.tif')
        with rasterio.open(out_name, 'w', **profile) as dst:
                dst.write(data.astype(rasterio.uint8), 1)
        
    print('Reclassified file:' + f'{out_name}')
    
    return data, out_name

In [21]:
grid, out_name =  reclassified(rasterized_vector, rst_fn, out_dir = out_dir)

/ncr2420/ANSU/6_Tasks/2110_ET/test/rasterize.tif
Reclassified file:/ncr2420/ANSU/6_Tasks/2110_ET/test/reclassfied_LC.tif
